# Load and Preprocess the data

In [73]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [74]:
df=pd.read_csv("Breast.csv")

In [75]:
df.shape

(96, 24483)

In [76]:
# Assign column names
df.columns = [f'feature_{i}' for i in range(df.shape[1] - 1)] + ['target']

In [77]:
df.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_24473', 'feature_24474', 'feature_24475', 'feature_24476',
       'feature_24477', 'feature_24478', 'feature_24479', 'feature_24480',
       'feature_24481', 'target'],
      dtype='object', length=24483)

In [78]:
df['target'].value_counts()

,count
target,
0,51
1,45


In [79]:
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_24473,feature_24474,feature_24475,feature_24476,feature_24477,feature_24478,feature_24479,feature_24480,feature_24481,target
0,2,-0.081,0.009,-0.091,-0.518,-0.502,-0.149,0.098,-0.090,0.138,...,-0.531,-0.020,0.014,-0.123,0.148,0.024,-0.070,-0.209,0.105,1
1,3,-0.125,0.070,-0.006,-0.575,-0.585,-0.183,0.102,0.023,-0.350,...,-0.883,-0.159,0.022,0.006,-0.086,0.019,0.026,-0.822,0.199,1
2,4,-0.270,0.123,0.056,-0.499,-0.402,-0.099,-0.145,-0.103,0.181,...,-0.044,-0.096,0.018,0.000,0.076,0.057,-0.016,-0.360,-0.038,1
3,5,-0.141,0.025,-0.031,-0.465,-0.533,-0.065,0.101,-0.008,-0.019,...,0.280,-0.088,0.043,0.207,-0.124,-0.041,-0.077,-0.432,-0.015,1
4,6,-0.149,0.068,-0.084,-0.557,-0.595,-0.062,-0.056,-0.009,0.031,...,-0.258,-0.135,-0.013,-0.045,-0.114,0.643,-0.162,-0.976,-0.078,1


In [80]:
# Separate features and target
X = df.drop('target', axis=1).values
y = df['target'].values

In [81]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [82]:
# Convert back to DataFrame with original feature names
X_scaled = pd.DataFrame(X_scaled, columns=df.columns[:-1])

In [83]:
# Train/test split
X_train_full, X_test_full, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [84]:
type(X_train_full)

pandas.core.frame.DataFrame

# Train model on all features

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [86]:
baseline_model = RandomForestClassifier(random_state=42)
start_time = time.time()
baseline_model.fit(X_train_full, y_train)
baseline_train_time = time.time() - start_time
y_pred_baseline = baseline_model.predict(X_test_full)

# Apply PSO to select optimal features


## PSO

**Qu’est-ce que PSO ?**

PSO (Particle Swarm Optimization) est un algorithme d’optimisation inspiré du comportement collectif des oiseaux ou des poissons lorsqu’ils cherchent de la nourriture.

Chaque particule représente une solution candidate dans l’espace de recherche (un sous-ensemble de caractéristiques dans notre cas).

Les particules se déplacent dans l’espace de recherche en étant influencées par :

* Leur propre meilleur souvenir personnel (meilleure position atteinte),

* Le meilleur souvenir du groupe (meilleure position trouvée par tout l’essaim).

**Principe de fonctionnement**

Chaque particule a :

* Une position (vecteur binaire pour la sélection de caractéristiques, ex: [1, 0, 1, 0] = sélectionner les features 0 et 2),

* Une vitesse (qui détermine la probabilité de changer de position),

* Une fonction de fitness qui évalue la qualité de la solution (ex: accuracy d’un modèle entraîné sur les features sélectionnées).

À chaque itération :
1. La fitness de chaque particule est évaluée.

2. On met à jour :

* Sa meilleure position personnelle (si la nouvelle fitness est meilleure),

* La meilleure position globale (parmi toutes les particules).

3. La vitesse et la position de chaque particule sont mises à jour en utilisant une formule mathématique.



**Hyperparametres de PSO**

* c1 (Cognitive coefficient)

  Role/influence: Apprentissage personnel

  Détails: Contrôle dans quelle mesure chaque particule est attirée par sa meilleure solution individuelle déjà trouvée. Un c1 plus élevé encourage une exploration individuelle.

* c2	(Social coefficient)

	Role/influence: Apprentissage collectif
  
  Détails: Influence de la meilleure solution globale (trouvée par le groupe). Un c2 élevé encourage les particules à se rapprocher de cette solution.

* w	(Inertia weight)

	Role/influence: Inertie du mouvement

  Détails:	Détermine à quel point la vitesse précédente influence le mouvement actuel. Un w élevé favorise l'exploration, un w plus bas favorise l'exploitation (raffinage).

* k	(Number of neighbors)

  Role/influence: 	Taille du voisinage (PSO local)

  Détails: Spécifie le nombre de particules voisines à prendre en compte pour décider du mouvement (dans les variantes où on ne prend pas toute la population).

* p	(Minkowski p-norm)

	Role/influence: Distance utilisée pour définir les voisins

  Détails: p=2 correspond à la distance euclidienne classique. Peut être ajusté pour d'autres types de distance si nécessaire.


## Code

In [87]:
!pip install pyswarms

In [88]:
import pyswarms as ps

In [89]:
# Define fitness function
def fitness_function(particles, alpha=0.9):
    scores = []
    for particle in particles:
      #mask : un tableau booléen indiquant quelles features sont sélectionnées (valeurs supérieures à 0.5 dans la particule → True).
        mask = particle > 0.5

        #Si aucune feature n’est sélectionnée, on donne un score pénalisant élevé (1.0) et on passe à la particule suivante. Cela évite d’entraîner un modèle vide.
        if np.count_nonzero(mask) == 0:
            scores.append(1.0)
            continue

        #Sélection des colonnes (features) activées pour entraîner et tester le modèle uniquement sur les colonnes sélectionnées.
        X_train_sel = X_train_full.iloc[:, mask]
        X_test_sel = X_test_full.iloc[:, mask]

        #Entraînement d’un modèle de forêt aléatoire sur les features sélectionnées.
        model = RandomForestClassifier(random_state=42)
        model.fit(X_train_sel, y_train)

        #Prédiction sur les données de test avec ces mêmes features.
        preds = model.predict(X_test_sel)

        #Calcul de l’erreur de classification (1 - accuracy).
        error = 1 - accuracy_score(y_test, preds)

        #Score de fitness :
        score = alpha * error + (1 - alpha) * (np.sum(mask) / X.shape[1])
        #On minimise cette fitness : meilleure performance + moins de features = meilleure solution.

        scores.append(score)
    return np.array(scores)

**NB**

alpha : paramètre de pondération entre la performance du modèle et le nombre de caractéristiques utilisées. (valeur par défaut 0.9 → priorité à la performance)

In [90]:
# PSO settings
dimensions = X.shape[1]
options = {
    'c1': 0.5,
    'c2': 0.3,
    'w': 0.9,
    'k': 5,       # number of neighbors
    'p': 2        # Minkowski p-norm
}
optimizer = ps.discrete.BinaryPSO(n_particles=10, dimensions=dimensions, options=options)

In [91]:
# Run optimization
cost, pos = optimizer.optimize(fitness_function, iters=10)
selected_features_default = np.where(pos > 0.5)[0]

2025-05-25 18:22:04,460 - pyswarms.discrete.binary - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 5, 'p': 2}
pyswarms.discrete.binary: 100%|██████████|10/10, best_cost=0.41
2025-05-25 18:23:14,872 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.4100571848705171, best pos: [1 1 0 ... 0 1 1]


#Train model on PSO-selected features

In [92]:
# Reduce feature set
X_train_pso = X_train_full.iloc[:, selected_features_default]
X_test_pso = X_test_full.iloc[:, selected_features_default]

In [93]:
pso_model = RandomForestClassifier(random_state=42)
start_time = time.time()
pso_model.fit(X_train_pso, y_train)
pso_train_time_default = time.time() - start_time
y_pred_pso_default = pso_model.predict(X_test_pso)

# Perform Grid Search to Optimize PSO Hyperparameters

In [94]:
search_space = [
    {'c1': 0.3, 'c2': 0.3, 'w': 0.5, 'k': 5, 'p': 2},
    {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 5, 'p': 2},
    {'c1': 0.8, 'c2': 0.8, 'w': 0.5, 'k': 5, 'p': 2},
]


In [95]:
best_score = float("inf")
best_pos = None
best_opt = None

for options in search_space:
    optimizer = ps.discrete.BinaryPSO(n_particles=10, dimensions=dimensions, options=options)
    score, position = optimizer.optimize(fitness_function, iters=10, verbose=False)
    if score < best_score:
        best_score = score
        best_pos = position
        best_opt = options

In [96]:
selected_features_optimized = np.where(best_pos > 0.5)[0]

# Save the Best Feature Subset and Train on Selected Features

In [98]:
# Use the feature names from the original DataFrame (excluding the target)
feature_names = df.columns[:-1]

# Get the names of the selected features
selected_feature_names = feature_names[selected_features_optimized]

# Save to CSV
pd.DataFrame(selected_feature_names).to_csv("best_features_pso.csv", index=False, header=False)

In [99]:
# Train on optimized PSO-selected features
X_train_opt = X_train_full.iloc[:, selected_features_optimized]
X_test_opt = X_test_full.iloc[:, selected_features_optimized]

In [100]:
optimized_model = RandomForestClassifier(random_state=42)
start_time = time.time()
optimized_model.fit(X_train_opt, y_train)
pso_train_time_opt = time.time() - start_time
y_pred_pso_opt = optimized_model.predict(X_test_opt)

# Evaluate all Scenarios

In [101]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [104]:
def evaluate_model(name, y_true, y_pred, train_time, n_features):
    print(f"\n Evaluation: {name}")
    print(f"Accuracy       : {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision      : {precision_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Recall         : {recall_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1 Score       : {f1_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Training Time  : {train_time:.4f} seconds")
    print(f"#Features Used : {n_features}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")

In [105]:
# Baseline (all features)
evaluate_model("Baseline (All Features)", y_test, y_pred_baseline, baseline_train_time, X.shape[1])


 Evaluation: Baseline (All Features)
Accuracy       : 0.4500
Precision      : 0.5119
Recall         : 0.5110
F1 Score       : 0.4486
Training Time  : 0.8610 seconds
#Features Used : 24482
Confusion Matrix:
[[5 2]
 [9 4]]


In [106]:
# PSO Default
evaluate_model("PSO Default", y_test, y_pred_pso_default, pso_train_time_default, len(selected_features_default))


 Evaluation: PSO Default
Accuracy       : 0.6000
Precision      : 0.6593
Recall         : 0.6593
F1 Score       : 0.6000
Training Time  : 0.5068 seconds
#Features Used : 12255
Confusion Matrix:
[[6 1]
 [7 6]]


In [107]:
# PSO Optimized
evaluate_model("PSO Optimized", y_test, y_pred_pso_opt, pso_train_time_opt, len(selected_features_optimized))


 Evaluation: PSO Optimized
Accuracy       : 0.6500
Precision      : 0.6875
Recall         : 0.6978
F1 Score       : 0.6491
Training Time  : 0.4495 seconds
#Features Used : 12353
Confusion Matrix:
[[6 1]
 [6 7]]


# Summary table

In [108]:
summary_df = pd.DataFrame({
    "Model": ["Baseline", "PSO Default", "PSO Optimized"],
    "Train Time (s)": [baseline_train_time, pso_train_time_default, pso_train_time_opt],
    "#Features": [X.shape[1], len(selected_features_default), len(selected_features_optimized)]
})

summary_df

,Model,Train Time (s),#Features
0,Baseline,0.860964,24482
1,PSO Default,0.506788,12255
2,PSO Optimized,0.449480,12353
